In [ ]:
import torch
torch.cuda.empty_cache()

In [ ]:
import numpy as np
from tensorflow import keras
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input, Conv2D, Dropout, Activation, UpSampling2D, GlobalMaxPooling2D, multiply
from tensorflow.keras.backend import max
from tensorflow.keras.optimizers import Adam
import Task1_handling
import openslide
import cv2
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.activations import relu

In [ ]:
from keras_unet_collection import models, losses
#!pip install tensorflow_addons
#import tensorflow_addons as tfa
import keras.backend as K
import tensorflow as tf

In [ ]:
data=np.load('../dataset/dataset_sorted_deleted_08.npz')
org_img=data['x']
layer3=data['y']
layer5=data['z']

y=np.stack((layer3,layer5),axis=-1)/255
X=org_img
X_norm=np.zeros(X.shape)

#normalizaiton
for i in range(len(X_norm)):
    X_norm[i]=keras.utils.normalize(X[i]/255,axis=-1)

  
#train,test split
from sklearn.model_selection import train_test_split
np.random.seed(seed=2021)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.2, shuffle=True)

In [ ]:
# mse=MeanSquaredError()

# def area_loss(y_true,y_pred):
#     y_pred_th=relu(y_pred,threshold=0.1)
#     return mse(y_true,y_pred_th)

In [ ]:
model =load_model('../pretrained/atten_unet_ResNet101.hdf5', compile=False)

model.summary()

In [ ]:
def dice_channel(y_true,y_pred):
    dice_3=losses.dice(y_true[:,:,:,0],y_pred[:,:,:,0])
    dice_5=losses.dice(y_true[:,:,:,1],y_pred[:,:,:,1])
    
    return (dice_3+dice_5)/2

def dice_layer3(y_true,y_pred):
    dice_3=losses.dice(y_true[:,:,:,0],y_pred[:,:,:,0])
    return dice_3

def dice_layer5(y_true,y_pred):
    dice_5=losses.dice(y_true[:,:,:,1],y_pred[:,:,:,1])
    return dice_5

In [ ]:
def dice_coef_channel(y_true,y_pred):
    dice_coef_3=losses.dice_coef(y_true[:,:,:,0],y_pred[:,:,:,0])
    dice_coef_5=losses.dice_coef(y_true[:,:,:,1],y_pred[:,:,:,1])
    
    return (1*dice_coef_3+dice_coef_5)/2

def dice_coef_layer3(y_true,y_pred):
    dice_coef_3=losses.dice_coef(y_true[:,:,:,0],y_pred[:,:,:,0])
    return dice_coef_3

def dice_coef_layer5(y_true,y_pred):
    dice_coef_5=losses.dice_coef(y_true[:,:,:,1],y_pred[:,:,:,1])
    return dice_coef_5

In [ ]:
from  tensorflow.keras.callbacks import ModelCheckpoint

checkpoint_filepath = './models/'
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=False,
    monitor='val_dice_coef_channel',
    mode='min',
    save_best_only=True)

In [ ]:
model.compile(loss=[keras.losses.BinaryCrossentropy(),losses.dice_coef],loss_weights=[0.5,0.5],optimizer=Adam(learning_rate = 1e-4), 
              metrics=['accuracy', dice_coef_channel,dice_coef_layer5])

In [ ]:
Unet_history = model.fit(X_train, y_train, 
                    verbose=1,
                    batch_size = 8,
                    validation_data=(X_test, y_test), 
                    shuffle=False,callbacks=[model_checkpoint_callback],
                    epochs=20)

In [ ]:
model.save('att_unet_bce+dice_channel_ResNet101.hdf5')

In [ ]:
#plot the training and validation accuracy and loss at each epoch
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

acc = history.history['dice_coef_channel']
#acc = history.history['accuracy']
val_acc = history.history['val_dice_coef_channel']
#val_acc = history.history['val_accuracy']

plt.plot(epochs, acc, 'y', label='Training Dice')
plt.plot(epochs, val_acc, 'r', label='Validation Dice')
plt.title('Training and validation Dice')
plt.xlabel('Epochs')
plt.ylabel('Dice')
plt.legend()
plt.show()